In [1]:
import torch
import pandas as pd
import numpy as np
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn import metrics
import torch.optim as optim

In [2]:
sub_features = ['42 tGravityAcc-mean()-Y',
 '43 tGravityAcc-mean()-Z',
 '51 tGravityAcc-max()-Y',
 '52 tGravityAcc-max()-Z',
 '54 tGravityAcc-min()-Y',
 '55 tGravityAcc-min()-Z',
 '56 tGravityAcc-sma()',
 '59 tGravityAcc-energy()-Z',
 '125 tBodyGyro-std()-Y',
 '128 tBodyGyro-mad()-Y',
 '138 tBodyGyro-energy()-Y',
 '165 tBodyGyroJerk-std()-Y',
 '168 tBodyGyroJerk-mad()-Y',
 '178 tBodyGyroJerk-energy()-Y',
 '181 tBodyGyroJerk-iqr()-Y',
 '425 fBodyGyro-mean()-Y',
 '428 fBodyGyro-std()-Y',
 '431 fBodyGyro-mad()-Y',
 '441 fBodyGyro-energy()-Y',
 '475 fBodyGyro-bandsEnergy()-1,8',
 '478 fBodyGyro-bandsEnergy()-25,32',
 '483 fBodyGyro-bandsEnergy()-1,16',
 '487 fBodyGyro-bandsEnergy()-1,24',
 '559 angle(X,gravityMean)',
 '560 angle(Y,gravityMean)',
 '561 angle(Z,gravityMean)']

act_features = ['4 tBodyAcc-std()-X',
 '7 tBodyAcc-mad()-X',
 '10 tBodyAcc-max()-X',
 '17 tBodyAcc-energy()-X',
 '202 tBodyAccMag-std()',
 '204 tBodyAccMag-max()',
 '215 tGravityAccMag-std()',
 '217 tGravityAccMag-max()',
 '266 fBodyAcc-mean()-X',
 '269 fBodyAcc-std()-X',
 '272 fBodyAcc-mad()-X',
 '275 fBodyAcc-max()-X',
 '282 fBodyAcc-energy()-X',
 '303 fBodyAcc-bandsEnergy()-1,8',
 '311 fBodyAcc-bandsEnergy()-1,16',
 '315 fBodyAcc-bandsEnergy()-1,24',
 '504 fBodyAccMag-std()',
 '505 fBodyAccMag-mad()',
 '506 fBodyAccMag-max()',
 '509 fBodyAccMag-energy()']

input_shape = len(sub_features) + len(act_features)

In [3]:
def classifier_block(input_dim, output_dim):
    return nn.Sequential(
        nn.Linear(input_dim, output_dim),
        nn.Dropout(0.1),
        nn.LeakyReLU(0.05)
    )

class Classifier(nn.Module):
    def __init__(self, feature_dim = input_shape):
        super(Classifier, self).__init__()
        self.network = nn.Sequential(
            classifier_block(feature_dim, 25),
            classifier_block(25, 20),
            classifier_block(20, 15),
            classifier_block(15, 10),
            nn.Linear(10, 9)
        )
    def forward(self, x):
        return self.network(x)

In [4]:
#defines each generator layer
#input and output dimensions needed
def generator_block(input_dim, output_dim):
    return nn.Sequential(
        nn.Linear(input_dim, output_dim),
        nn.Dropout(0.1),
        nn.BatchNorm1d(output_dim),
        nn.ReLU(inplace = True)
    )

#returns n_samples of z_dim (number of dimensions of latent space) noise
def get_noise(n_samples, z_dim):
    return torch.randn(n_samples, z_dim)

#defines generator class
class Generator(nn.Module):
    def __init__(self, z_dim = 10, feature_dim = input_shape, hidden_dim = 128):
        super(Generator, self).__init__()
        self.gen = nn.Sequential(
            generator_block(z_dim, 80),
            generator_block(80, 60),
            generator_block(60, 50),
            nn.Linear(50, feature_dim),
            nn.Tanh()
        )
    def forward(self, noise):
        return self.gen(noise)

def load_model(model, model_name):
    model.load_state_dict(torch.load(f'../saved_models/{model_name}'))

In [5]:
#label is a list of integers specifying which labels to filter by
#users is a list of integers specifying which users to filter by
#y_label is a string, either "Activity" or "Subject" depending on what y output needs to be returned
def start_data(label, users, y_label, sub_features, act_features):
    #get the dataframe column names
    name_dataframe = pd.read_csv('../data/features.txt', delimiter = '\n', header = None)
    names = name_dataframe.values.tolist()
    names = [k for row in names for k in row] #List of column names

    data = pd.read_csv('../data/X_train.txt', delim_whitespace = True, header = None) #Read in dataframe
    data.columns = names #Setting column names
    
    X_train_1 = data[sub_features]
    X_train_2 = data[act_features]
    X_train = pd.concat([X_train_1, X_train_2], axis = 1)
    
    y_train_activity = pd.read_csv('../data/y_train.txt', header = None)
    y_train_activity.columns = ['Activity']
    
    y_train_subject = pd.read_csv('../data/subject_train.txt', header = None)
    y_train_subject.columns = ['Subject']

    GAN_data = pd.concat([X_train, y_train_activity, y_train_subject], axis = 1)
    GAN_data = GAN_data[GAN_data['Activity'].isin(label)]
    GAN_data = GAN_data[GAN_data['Subject'].isin(users)]
    
    X_1 = GAN_data[(GAN_data['Subject'] == 1) & (GAN_data['Activity'] == 1)].iloc[:,:-2].values
    X_2 = GAN_data[(GAN_data['Subject'] == 1) & (GAN_data['Activity'] == 3)].iloc[:,:-2].values
    X_3 = GAN_data[(GAN_data['Subject'] == 1) & (GAN_data['Activity'] == 4)].iloc[:,:-2].values
    X_4 = GAN_data[(GAN_data['Subject'] == 3) & (GAN_data['Activity'] == 1)].iloc[:,:-2].values
    X_5 = GAN_data[(GAN_data['Subject'] == 3) & (GAN_data['Activity'] == 3)].iloc[:,:-2].values
    X_6 = GAN_data[(GAN_data['Subject'] == 3) & (GAN_data['Activity'] == 4)].iloc[:,:-2].values
    X_7 = GAN_data[(GAN_data['Subject'] == 5) & (GAN_data['Activity'] == 1)].iloc[:,:-2].values
    X_8 = GAN_data[(GAN_data['Subject'] == 5) & (GAN_data['Activity'] == 3)].iloc[:,:-2].values
    X_9 = GAN_data[(GAN_data['Subject'] == 5) & (GAN_data['Activity'] == 4)].iloc[:,:-2].values
    
    X_train = np.concatenate((X_1, X_2, X_3, X_4, X_5, X_6, X_7, X_8, X_9))
    y_train = [0] * len(X_1) + [1] * len(X_2) + [2] * len(X_3) + [3] * len(X_4) + [4] * len(X_5) + [5] * len(X_6) + [6] * len(X_7) + [7] * len(X_8) + [8] * len(X_9)
    
    return X_train, np.asarray(y_train)

In [6]:
gen = Generator(z_dim = 100)
gen.eval()
load_model(gen, "U0A0 Solo GAN_gen.param")
latent_vectors = get_noise(35, 100)
fake_features_1 = gen(latent_vectors).detach().numpy()

y_1 = np.zeros(35)

gen = Generator(z_dim = 100)
gen.eval()
load_model(gen, "U0A1 Solo GAN_gen.param")
latent_vectors = get_noise(35, 100)
fake_features_2 = gen(latent_vectors).detach().numpy()

y_2 = np.ones(35)

gen = Generator(z_dim = 100)
gen.eval()
load_model(gen, "U0A2 Solo GAN_gen.param")
latent_vectors = get_noise(35, 100)
fake_features_3 = gen(latent_vectors).detach().numpy()

y_3 = np.ones(35) + 1

gen = Generator(z_dim = 100)
gen.eval()
load_model(gen, "U1A0 Solo GAN_gen.param")
latent_vectors = get_noise(35, 100)
fake_features_4 = gen(latent_vectors).detach().numpy()

y_4 = np.ones(35) + 2

gen = Generator(z_dim = 100)
gen.eval()
load_model(gen, "U1A1 Solo GAN_gen.param")
latent_vectors = get_noise(35, 100)
fake_features_5 = gen(latent_vectors).detach().numpy()

y_5 = np.ones(35) + 3

gen = Generator(z_dim = 100)
gen.eval()
load_model(gen, "U1A2 Solo GAN_gen.param")
latent_vectors = get_noise(35, 100)
fake_features_6 = gen(latent_vectors).detach().numpy()

y_6 = np.ones(35) + 4

gen = Generator(z_dim = 100)
gen.eval()
load_model(gen, "U2A0 Solo GAN_gen.param")
latent_vectors = get_noise(35, 100)
fake_features_7 = gen(latent_vectors).detach().numpy()

y_7 = np.ones(35) + 5

gen = Generator(z_dim = 100)
gen.eval()
load_model(gen, "U2A1 Solo GAN_gen.param")
latent_vectors = get_noise(35, 100)
fake_features_8 = gen(latent_vectors).detach().numpy()

y_8 = np.ones(35) + 6

gen = Generator(z_dim = 100)
gen.eval()
load_model(gen, "U2A2 Solo GAN_gen.param")
latent_vectors = get_noise(35, 100)
fake_features_9 = gen(latent_vectors).detach().numpy()

y_9 = np.ones(35) + 7

X_train = np.concatenate((fake_features_1, fake_features_2, fake_features_3, fake_features_4, fake_features_5, fake_features_6,
                         fake_features_7, fake_features_8, fake_features_9))
y_train = np.concatenate((y_1, y_2, y_3, y_4, y_5, y_6, y_7, y_8, y_9))

In [7]:
activities = [1, 3, 4]
users = [1, 3, 5]
X_test, y_test = start_data(activities, users, "Activity", sub_features, act_features)
_, X_test, _, y_test = train_test_split(X_test, y_test, test_size = 0.2, shuffle = True)

In [8]:
model = Classifier()
lr = 0.001
n_epochs = 5000
batch_size = 250

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = lr)

train_features = torch.tensor(X_train)
train_labels = torch.tensor(y_train)
test_features = torch.tensor(X_test)
test_labels = torch.tensor(y_test)

train_data = torch.utils.data.TensorDataset(train_features, train_labels)
test_data = torch.utils.data.TensorDataset(test_features, test_labels)

train_loader = torch.utils.data.DataLoader(train_data, batch_size = batch_size, shuffle = True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size = len(test_labels), shuffle = True)

In [9]:
for epoch in range(n_epochs):
    total_loss = 0
    for batch in train_loader:
        features, labels = batch
        
        optimizer.zero_grad()
        preds = model(features.float())
        
        loss = criterion(preds, labels.long()) 
        loss.backward()
        
        optimizer.step()
        total_loss += loss.item()
        
    print(f'Epoch {epoch + 1}, Loss: {total_loss}, Final Batch Loss: {loss.item()}')

Epoch 1, Loss: 4.4255921840667725, Final Batch Loss: 2.2105813026428223
Epoch 2, Loss: 4.429545640945435, Final Batch Loss: 2.2171683311462402
Epoch 3, Loss: 4.417402267456055, Final Batch Loss: 2.2021896839141846
Epoch 4, Loss: 4.417039394378662, Final Batch Loss: 2.2066190242767334
Epoch 5, Loss: 4.378518104553223, Final Batch Loss: 2.1561367511749268
Epoch 6, Loss: 4.3835296630859375, Final Batch Loss: 2.164398431777954
Epoch 7, Loss: 4.4138922691345215, Final Batch Loss: 2.205383062362671
Epoch 8, Loss: 4.407275676727295, Final Batch Loss: 2.1974239349365234
Epoch 9, Loss: 4.426706790924072, Final Batch Loss: 2.2281370162963867
Epoch 10, Loss: 4.435963869094849, Final Batch Loss: 2.242360830307007
Epoch 11, Loss: 4.393233060836792, Final Batch Loss: 2.185807943344116
Epoch 12, Loss: 4.405672073364258, Final Batch Loss: 2.2059266567230225
Epoch 13, Loss: 4.412482738494873, Final Batch Loss: 2.2161779403686523
Epoch 14, Loss: 4.378667116165161, Final Batch Loss: 2.17116379737854
Epoc

Epoch 123, Loss: 2.207185387611389, Final Batch Loss: 1.1145744323730469
Epoch 124, Loss: 2.1200754642486572, Final Batch Loss: 1.0508846044540405
Epoch 125, Loss: 2.224388837814331, Final Batch Loss: 1.1398764848709106
Epoch 126, Loss: 2.1321444511413574, Final Batch Loss: 1.027369499206543
Epoch 127, Loss: 2.199636697769165, Final Batch Loss: 1.071033000946045
Epoch 128, Loss: 2.259600281715393, Final Batch Loss: 1.183103322982788
Epoch 129, Loss: 2.1562068462371826, Final Batch Loss: 1.0377159118652344
Epoch 130, Loss: 2.101579785346985, Final Batch Loss: 1.065350890159607
Epoch 131, Loss: 2.298153519630432, Final Batch Loss: 1.2246284484863281
Epoch 132, Loss: 2.0777549743652344, Final Batch Loss: 1.0589368343353271
Epoch 133, Loss: 2.308066964149475, Final Batch Loss: 1.2149765491485596
Epoch 134, Loss: 2.009783446788788, Final Batch Loss: 0.9373393654823303
Epoch 135, Loss: 2.21931791305542, Final Batch Loss: 1.1587544679641724
Epoch 136, Loss: 2.075596332550049, Final Batch Loss

Epoch 259, Loss: 1.7952112555503845, Final Batch Loss: 0.92955082654953
Epoch 260, Loss: 1.8047552108764648, Final Batch Loss: 0.9139426946640015
Epoch 261, Loss: 1.7471272945404053, Final Batch Loss: 0.8749401569366455
Epoch 262, Loss: 1.7031832337379456, Final Batch Loss: 0.7729291915893555
Epoch 263, Loss: 1.7932570576667786, Final Batch Loss: 0.9264013171195984
Epoch 264, Loss: 1.730264961719513, Final Batch Loss: 0.8632463812828064
Epoch 265, Loss: 1.8900806903839111, Final Batch Loss: 0.979037344455719
Epoch 266, Loss: 1.81007319688797, Final Batch Loss: 0.9451936483383179
Epoch 267, Loss: 1.7333170771598816, Final Batch Loss: 0.8756592869758606
Epoch 268, Loss: 1.872378647327423, Final Batch Loss: 0.9894999861717224
Epoch 269, Loss: 1.6645959615707397, Final Batch Loss: 0.755240261554718
Epoch 270, Loss: 1.6754107475280762, Final Batch Loss: 0.864798903465271
Epoch 271, Loss: 1.775054395198822, Final Batch Loss: 0.9081486463546753
Epoch 272, Loss: 1.774889349937439, Final Batch 

Epoch 394, Loss: 1.401848554611206, Final Batch Loss: 0.7440906763076782
Epoch 395, Loss: 1.4374247789382935, Final Batch Loss: 0.7680925726890564
Epoch 396, Loss: 1.4570482969284058, Final Batch Loss: 0.7705191373825073
Epoch 397, Loss: 1.4322638511657715, Final Batch Loss: 0.7582924962043762
Epoch 398, Loss: 1.3671597242355347, Final Batch Loss: 0.6597623825073242
Epoch 399, Loss: 1.3407734036445618, Final Batch Loss: 0.6622685194015503
Epoch 400, Loss: 1.2895036935806274, Final Batch Loss: 0.590908408164978
Epoch 401, Loss: 1.2803862690925598, Final Batch Loss: 0.6158269643783569
Epoch 402, Loss: 1.3187863230705261, Final Batch Loss: 0.6608487963676453
Epoch 403, Loss: 1.2820913195610046, Final Batch Loss: 0.5788602828979492
Epoch 404, Loss: 1.3974415063858032, Final Batch Loss: 0.7759535312652588
Epoch 405, Loss: 1.3116986155509949, Final Batch Loss: 0.6659000515937805
Epoch 406, Loss: 1.3567039966583252, Final Batch Loss: 0.7509855031967163
Epoch 407, Loss: 1.3893995881080627, Fin

Epoch 533, Loss: 1.1340494751930237, Final Batch Loss: 0.5706457495689392
Epoch 534, Loss: 1.0958423614501953, Final Batch Loss: 0.5231528282165527
Epoch 535, Loss: 0.9740162789821625, Final Batch Loss: 0.453858882188797
Epoch 536, Loss: 1.1685205698013306, Final Batch Loss: 0.6289130449295044
Epoch 537, Loss: 1.150349497795105, Final Batch Loss: 0.5815105438232422
Epoch 538, Loss: 1.1717252731323242, Final Batch Loss: 0.6271188855171204
Epoch 539, Loss: 1.1246063113212585, Final Batch Loss: 0.6000583171844482
Epoch 540, Loss: 1.1099607348442078, Final Batch Loss: 0.529817521572113
Epoch 541, Loss: 1.084691047668457, Final Batch Loss: 0.5215722322463989
Epoch 542, Loss: 1.1573519706726074, Final Batch Loss: 0.6361480355262756
Epoch 543, Loss: 0.9627979695796967, Final Batch Loss: 0.42465832829475403
Epoch 544, Loss: 1.2685922384262085, Final Batch Loss: 0.7114410400390625
Epoch 545, Loss: 0.9891577363014221, Final Batch Loss: 0.4841845631599426
Epoch 546, Loss: 1.0050953328609467, Fina

...

Epoch 4511, Loss: 0.04159349016845226, Final Batch Loss: 0.02800159342586994
Epoch 4512, Loss: 0.07929855212569237, Final Batch Loss: 0.051980145275592804
Epoch 4513, Loss: 0.02592936623841524, Final Batch Loss: 0.005164708010852337
Epoch 4514, Loss: 0.09209022764116526, Final Batch Loss: 0.08190193772315979
Epoch 4515, Loss: 0.02724501071497798, Final Batch Loss: 0.00683834170922637
Epoch 4516, Loss: 0.12409013509750366, Final Batch Loss: 0.044004082679748535
Epoch 4517, Loss: 0.0689233485609293, Final Batch Loss: 0.04083585366606712
Epoch 4518, Loss: 0.16732782870531082, Final Batch Loss: 0.10474240779876709
Epoch 4519, Loss: 0.1098401602357626, Final Batch Loss: 0.01712018810212612
Epoch 4520, Loss: 0.06665358319878578, Final Batch Loss: 0.03798580542206764
Epoch 4521, Loss: 0.03636162914335728, Final Batch Loss: 0.02348567359149456
Epoch 4522, Loss: 0.04370688833296299, Final Batch Loss: 0.019681235775351524
Epoch 4523, Loss: 0.025729963555932045, Final Batch Loss: 0.01448787283152

Epoch 4621, Loss: 0.03283082786947489, Final Batch Loss: 0.018263371661305428
Epoch 4622, Loss: 0.05634252727031708, Final Batch Loss: 0.013628244400024414
Epoch 4623, Loss: 0.038352878764271736, Final Batch Loss: 0.01952105574309826
Epoch 4624, Loss: 0.10893730632960796, Final Batch Loss: 0.09993964433670044
Epoch 4625, Loss: 0.0559863205999136, Final Batch Loss: 0.022603338584303856
Epoch 4626, Loss: 0.04248453862965107, Final Batch Loss: 0.02316303737461567
Epoch 4627, Loss: 0.03307504393160343, Final Batch Loss: 0.018293695524334908
Epoch 4628, Loss: 0.013854455202817917, Final Batch Loss: 0.006856279913336039
Epoch 4629, Loss: 0.05261869169771671, Final Batch Loss: 0.03006810136139393
Epoch 4630, Loss: 0.03687780164182186, Final Batch Loss: 0.01433584839105606
Epoch 4631, Loss: 0.04131005937233567, Final Batch Loss: 0.00632478529587388
Epoch 4632, Loss: 0.02832372486591339, Final Batch Loss: 0.004361458122730255
Epoch 4633, Loss: 0.03811363875865936, Final Batch Loss: 0.0165882185

Epoch 4731, Loss: 0.05178777314722538, Final Batch Loss: 0.019146865233778954
Epoch 4732, Loss: 0.08316313102841377, Final Batch Loss: 0.06457648426294327
Epoch 4733, Loss: 0.04921284411102533, Final Batch Loss: 0.012056873179972172
Epoch 4734, Loss: 0.023634139448404312, Final Batch Loss: 0.0021057967096567154
Epoch 4735, Loss: 0.04732196964323521, Final Batch Loss: 0.02265194058418274
Epoch 4736, Loss: 0.05095309764146805, Final Batch Loss: 0.028253359720110893
Epoch 4737, Loss: 0.05175399221479893, Final Batch Loss: 0.028751207515597343
Epoch 4738, Loss: 0.05224475637078285, Final Batch Loss: 0.01902935653924942
Epoch 4739, Loss: 0.051930627785623074, Final Batch Loss: 0.00871309544891119
Epoch 4740, Loss: 0.04742551036179066, Final Batch Loss: 0.0277960617095232
Epoch 4741, Loss: 0.060830400325357914, Final Batch Loss: 0.013713126070797443
Epoch 4742, Loss: 0.040632705204188824, Final Batch Loss: 0.015311061404645443
Epoch 4743, Loss: 0.0349190179258585, Final Batch Loss: 0.0082205

Epoch 4844, Loss: 0.022891396190971136, Final Batch Loss: 0.0043238489888608456
Epoch 4845, Loss: 0.02894625812768936, Final Batch Loss: 0.010599156841635704
Epoch 4846, Loss: 0.020608361810445786, Final Batch Loss: 0.006574421189725399
Epoch 4847, Loss: 0.04963554348796606, Final Batch Loss: 0.003092731349170208
Epoch 4848, Loss: 0.028460773639380932, Final Batch Loss: 0.01227936428040266
Epoch 4849, Loss: 0.02216820791363716, Final Batch Loss: 0.007952241227030754
Epoch 4850, Loss: 0.06819468596950173, Final Batch Loss: 0.006633235607296228
Epoch 4851, Loss: 0.05822578817605972, Final Batch Loss: 0.021890047937631607
Epoch 4852, Loss: 0.05504034645855427, Final Batch Loss: 0.033318180590867996
Epoch 4853, Loss: 0.03775767423212528, Final Batch Loss: 0.012672888115048409
Epoch 4854, Loss: 0.04033508989959955, Final Batch Loss: 0.01115360390394926
Epoch 4855, Loss: 0.10537439212203026, Final Batch Loss: 0.07588862627744675
Epoch 4856, Loss: 0.08427033387124538, Final Batch Loss: 0.0762

Epoch 4951, Loss: 0.024468901567161083, Final Batch Loss: 0.013794058002531528
Epoch 4952, Loss: 0.025354809127748013, Final Batch Loss: 0.010884663090109825
Epoch 4953, Loss: 0.04976326413452625, Final Batch Loss: 0.016536524519324303
Epoch 4954, Loss: 0.04089111275970936, Final Batch Loss: 0.02004464529454708
Epoch 4955, Loss: 0.021038567647337914, Final Batch Loss: 0.012227213010191917
Epoch 4956, Loss: 0.061504163313657045, Final Batch Loss: 0.05406690761446953
Epoch 4957, Loss: 0.023931321687996387, Final Batch Loss: 0.008213036693632603
Epoch 4958, Loss: 0.06736332178115845, Final Batch Loss: 0.058160267770290375
Epoch 4959, Loss: 0.03411849564872682, Final Batch Loss: 0.0027697307523339987
Epoch 4960, Loss: 0.025328420335426927, Final Batch Loss: 0.002936318749561906
Epoch 4961, Loss: 0.0910960454493761, Final Batch Loss: 0.08091916143894196
Epoch 4962, Loss: 0.03879686817526817, Final Batch Loss: 0.02848508022725582
Epoch 4963, Loss: 0.014544368255883455, Final Batch Loss: 0.00

In [83]:
softmax = nn.Softmax(dim = 1)
for batch in test_loader:
    features, labels = batch
    _, preds = torch.max(softmax(model(features.float())), dim = 1)
    print(metrics.confusion_matrix((labels).cpu(), preds.cpu()))
    print(metrics.classification_report((labels).cpu(), preds.cpu(), digits = 3))

[[17  1  0  0  0  0  0  0  0]
 [ 0 12  0  0  1  0  0  0  0]
 [ 0  0  8  0  0  0  0  0  1]
 [ 0  0  0 14  0  0  0  0  0]
 [ 0  0  0  0 12  0  0  0  0]
 [ 0  0  1  0  0  3  0  0  8]
 [ 1  0  0  0  0  1 10  0  0]
 [ 0  0  0  0  0  0  0  6  0]
 [ 0  0  0  0  0  1  0  0  3]]
              precision    recall  f1-score   support

           0      0.944     0.944     0.944        18
           1      0.923     0.923     0.923        13
           2      0.889     0.889     0.889         9
           3      1.000     1.000     1.000        14
           4      0.923     1.000     0.960        12
           5      0.600     0.250     0.353        12
           6      1.000     0.833     0.909        12
           7      1.000     1.000     1.000         6
           8      0.250     0.750     0.375         4

    accuracy                          0.850       100
   macro avg      0.837     0.843     0.817       100
weighted avg      0.883     0.850     0.852       100

